In [1]:
import pandas as pd
import scipy
import numpy as np
import math

# Data

## Raw Data

In [3]:
#World Hapiness Data
wh_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/WH_data_2019.csv")

#Women's Prosperity Index
wps_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/WPS-Index-2021-Data.csv", 
                       encoding = 'unicode_escape')

#Tropical Climate Data
tropical_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/tropical_countries.csv")

#climate data
climate_data = pd.read_csv('https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/climate_zones.csv')

#LGBTQ Safety and Welfare Data
lgbtq_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/LGBTQ_Safety_Index.csv")

#Industry Sector Data
sector_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/country_sectors.csv")

#City Climate Data
city_data = pd.read_csv("https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/city_data.csv")

#air quality Data
aq_data = pd.read_csv('https://raw.githubusercontent.com/ereppond/CSE6242-GroupProject/main/BlissfulBorders/data/SEDAC_annual_pm2-5_concentration.csv')


## Clean Data

In [4]:
#Some column renaming to prepare for joins

#Clean World Happiness Data (preserving all vars)
wh_data.rename(columns = {'Overall rank': 'WH Rank', 
                          'Country or region': 'Country',
                          'Score': 'WH Score'}, inplace = True)

#Clean Women's Prosperity Index Data
wps_data.rename(columns = {'ï»¿WPS Index rank': 'WPS Rank', 
                           'WPS Index score': 'WPS Score', 
                           'Country':'WPS Country'}, inplace = True)
wps_data = wps_data.iloc[:,0:3] #select vars

#Clean Tropical climate data
climate_data.rename(columns = {'Country':'Climate Country'}, inplace = True)
climate_data['Climate Country'] = climate_data['Climate Country'].str.strip() #removing leading spaces

#Clean LGBTQ data
lgbtq_data.rename(columns = {'Rank': 'LGBTQ Rank', 
                             "Score \n(worst is -100)\n(best is 0)": 'LGBTQ Score',
                             'Grade': 'LGBTQ Grade',
                             'Country': 'LGBTQ Country'}, inplace = True)

lgbtq_data = lgbtq_data[['LGBTQ Rank', 'LGBTQ Country', 'LGBTQ Score', 'LGBTQ Grade']]

#Clean sector data
sector_data.rename(columns = {'Country': 'Sector Country'}, inplace = True)
sector_data['Sector Country'] = sector_data['Sector Country'].str.strip() #removing leading spaces

## Joins

In [5]:
#since the World Happiness Index is the objectove value for this application, 
#wh_data is the left df on whihc the df is started which ensures all countries in the WHI are included
#subsequent joins eliminate observations that are not in the WHI

#merge city_data with World Happiness 
df = city_data.merge(wh_data,
                    how = 'left',
                    left_on = "country",
                    right_on = "Country")

#merge Women's Prosperity df
df = df.merge(wps_data,
                   how = 'left', 
                   left_on = "country",
                   right_on = "WPS Country")

#merge LGBTQ data with main df
df = df.merge(lgbtq_data,
              how = 'left',
              left_on = 'country',
              right_on = 'LGBTQ Country')

#merge climate type data with main df
df = df.merge(climate_data,
              how = 'left',
              left_on = 'country',
              right_on = 'Climate Country')

#merge economic sector data data with main df
df = df.merge(sector_data,
              how = 'left',
              left_on = 'country',
              right_on = 'Sector Country')

#merge air quality data
df = df.merge(aq_data,
           how = 'left',
            left_on = ["city_ascii", "Country"],
            right_on = ["NAME","COUNTRYENG"] )

#drop duplicate columns
df.drop(['Climate Country', 'WPS Country', 'Avg_temp_F', 'Avg_temp_C',
       'Sector Country','WPS Country','Sector Country','Sector Country','AVPMU_1998',
       'AVPMU_1999', 'AVPMU_2000', 'AVPMU_2001', 'AVPMU_2002', 'AVPMU_2003',
      'AVPMU_2004', 'AVPMU_2005', 'AVPMU_2006', 'AVPMU_2007', 'AVPMU_2008',
       'AVPMU_2009', 'AVPMU_2010', 'AVPMU_2011', 'AVPMU_2012', 'AVPMU_2013',
       'AVPMU_2014', 'AVPMU_2015'], axis=1, inplace=True) 



## Transform Values

In [6]:
#assign int values to LGBTQ letter grades
grades = list(df['LGBTQ Grade'].unique())
grades = [grade for grade in grades if type(grade) == str] #only letter grades
values = sorted(list(range(1,len(grades))), reverse=True) # list of integers in reverse
scores = dict(zip(grades, values))
df['LGBTQ Score'] = df['LGBTQ Grade'].apply(lambda x: scores.get(x)) #new column with inter values for grades


In [7]:
#additing climate infomation detail
climate_codes = df['Climate zone'].unique()

climate_zones = {
                "AF": ["Tropical rainforest", "Tropical"],
                "AM": ["Tropical monsoon", "Tropical"],
                "AW": ["Tropical wet and dry or savanna", "Tropical"],
                "BWH": ["Subtropical desert", "Arid"],
                "BSH": ["Subtropical steppe", "Arid"],
                "BSK": ["Mid-latitude steppe", "Arid"],
                "BWK": ["Mid-latitude desert", "Arid"],
                "CFA": ["Humid subtropical, no dry season", "Temperate"],
                "CWA": ["Humid subtropical, dry winter", "Temperate"],
                "CSA": ["Mediterranean, hot summer", "Temperate"],
                "CSB": ["Mediterranean, warm summer", "Temperate"],
                "CWB": ["Temperate highland tropical climate with dry winters", "Temperate"],
                "DFB": ["Humid continental, no dry season, warm summer", "Cold"],
                "DFC": ["Subartic, severe winter, no dry season, cool summer", "Cold"],
                "DSC": ["Humid continental, dry warm summer", "Cold"],
                "DSB": ["Humid continental, dry warm summer", "Cold"],
                "DWA": ["Humid continental, severe dry winter, hot summer", "Cold"],
                "DWB": ["Humid continental, severe dry winter, warm summer", "Cold"],
                "DWC": ["Subartic, dry winter, cool summer", "Cold"],
                "ET": ["Tundra", "Cold"]
                 }

def map_climate_zones(zone):
    """
    Maps descriptions found in climate_zones dict to climate code in df per dict
    """
    if zone in climate_zones:
        return climate_zones[zone]
    else:
        return ["", ""]

# apply the mapping function to the climate zone column and create two new columns for descriptions
df[["Climate description", "Climate type"]] = df["Climate zone"].apply(map_climate_zones).tolist()

In [8]:
#convert sector data to decimal
df['Agricultural percent'] = df['Agricultural percent'].str.rstrip('%').astype('float') / 100.0
df['Industrial percent'] = df['Industrial percent'].str.rstrip('%').astype('float') / 100.0
df['Service percent'] = df['Service percent'].str.rstrip('%').astype('float') / 100.0

#add dominant sector
df["dom_sector"] = df[['Agricultural percent','Industrial percent','Service percent']].idxmax(axis=1)
df["dom_sector"] = df["dom_sector"].str.replace(' percent', '')

In [9]:
#creates humidity category
def categorize_humidity(humidity):
    if humidity > 60:
        return "Humid"
    elif humidity > 29:
        return "Medium"
    else:
        return "Dry"

# apply the function to the 'avg_humidity' column and create a new column with the results
df['humidity'] = df['avg_humidity'].apply(lambda x: categorize_humidity(x))

In [10]:
df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,...,SQKM,COUNTRYENG,ISO3,UNSDCODE,CONTINENT,AVPMU_2016,Climate description,Climate type,dom_sector,humidity
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,...,43605.800780,Japan,JPN,392.0,Asia,11.96,"Humid continental, no dry season, warm summer",Cold,Service,Humid
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,...,4104.859863,Indonesia,IDN,360.0,Asia,30.22,Tropical rainforest,Tropical,Service,Humid
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tropical wet and dry or savanna,Tropical,Service,Humid
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,...,2234.040039,Philippines,PHL,608.0,Asia,12.66,Tropical rainforest,Tropical,Service,Humid
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tropical wet and dry or savanna,Tropical,Service,Humid


In [11]:
#creates Air quality category based on most recent (2016) PM 2.5 value

def determine_air_quality(value):
    if math.isnan(value):
        return 'NaN'
    elif value <= 50:
        return 'Good'
    elif 51 <= value <= 100:
        return 'Moderate'
    elif 101 <= value <= 150:
        return 'Unhealthy for sensitive groups'
    elif 151 <= value <= 200:
        return 'Unhealthy'
    elif 201 <= value <= 300:
        return 'Very unhealthy'
    else:
        return 'Hazardous'

# Apply the function to create a new 'air_quality' column
df['air_quality'] = df['AVPMU_2016'].apply(determine_air_quality)

# change #2016 aq values to negative for weighted sum calculation
df['neg_aq'] = (df['AVPMU_2016'] * -1)

In [12]:
#create cataegories for population size

def determine_city_size(value):
    if math.isnan(value):
        return 'NaN'
    elif value <= 4999:
        return 'Rural'
    elif 5000 <= value <= 49999:
        return 'Town'
    elif 50000 <= value <= 99999:
        return 'City'
    else:
        return 'Big City'

# Apply the function to create a new 'city_size' column
df['city_size'] = df['population'].apply(determine_city_size)

In [13]:
#Normalizing data to range 0,1
df['LGBTQ_norm'] = (df['LGBTQ Score'] - df['LGBTQ Score'].min()) / (df['LGBTQ Score'].max() - df['LGBTQ Score'].min())
df['WPS_norm'] = (df['WPS Score'] - df['WPS Score'].min()) / (df['WPS Score'].max() - df['WPS Score'].min())
df['Freedom_norm'] = (df['Freedom to make life choices'] - df['Freedom to make life choices'].min()) / (df['Freedom to make life choices'].max() - df['Freedom to make life choices'].min())
df['GDP_norm'] = (df['GDP per capita'] - df['GDP per capita'].min()) / (df['GDP per capita'].max() - df['GDP per capita'].min())
df['AQ_norm'] = (df['neg_aq'] - df['neg_aq'].min()) / (df['neg_aq'].max() - df['neg_aq'].min())
#df['POP_norm'] = (df['population'] - df['population'].min()) / (df['population'].max() - df['population'].min())

In [14]:
df.head(30)

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,...,dom_sector,humidity,air_quality,neg_aq,city_size,LGBTQ_norm,WPS_norm,Freedom_norm,GDP_norm,AQ_norm
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,...,Service,Humid,Good,-11.96,Big City,1.0,0.846273,0.705230,0.788005,0.886002
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,...,Service,Humid,Good,-30.22,Big City,0.9,0.666149,0.778130,0.552850,0.718939
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,...,Service,Humid,NaN,NaN,Big City,0.8,0.495342,0.789223,0.448337,NaN
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,...,Service,Humid,Good,-12.66,Big City,0.7,0.745342,0.884311,0.479216,0.879597
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,...,Service,Humid,NaN,NaN,Big City,0.6,0.708075,0.618067,0.596200,NaN
5,Seoul,Seoul,37.5600,126.9900,South Korea,KR,KOR,Seoul,primary,22394000.0,...,Service,Humid,NaN,NaN,Big City,NaN,0.852484,0.251981,0.772565,NaN
6,Seoul,Seoul,37.5600,126.9900,South Korea,KR,KOR,Seoul,primary,22394000.0,...,Industrial,Humid,NaN,NaN,Big City,NaN,0.852484,0.251981,0.772565,NaN
7,Mumbai,Mumbai,19.0758,72.8775,India,IN,IND,Mahārāshtra,admin,22186000.0,...,Service,Humid,NaN,NaN,Big City,0.8,0.495342,0.789223,0.448337,NaN
8,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22118000.0,...,Service,Humid,Moderate,-52.63,Big City,0.5,0.689441,0.825674,0.611045,0.513907
9,Mexico City,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,21505000.0,...,Service,Humid,Good,-20.33,Big City,0.4,0.694099,0.686212,0.635392,0.809424


# Weighted Sum Optimization Function

In [15]:
def optimize(df, user_profile, n=5):
    """
    Uses values from user_profile to filter df on users prefered climate, humidity level, 
    and dominant economic sector.
    Then uses uses ranked vars in user_profile to calculate weights for each var.
    Column is added to df with weighted value for each var.
    """
    # Filter for climate and sector
    df = df[(df['Climate type'] == user_profile["climate"]) & \
            (df['dom_sector'] == user_profile["sector"]) & \
            (df['city_size'] == user_profile["city_size"])].copy()
    
    # Normalize the ranks so that they sum up to 1
    rank_sum = user_profile["LGBTQ_rank"] +\
                user_profile["WPSI_rank"] + \
                user_profile["freedom_rank"] + \
                user_profile["GDP_rank"]
    LGBTQ_weight = user_profile["LGBTQ_rank"] / rank_sum
    WPS_weight = user_profile["WPSI_rank"] / rank_sum
    freedom_weight = user_profile["freedom_rank"] / rank_sum
    GDP_weight = user_profile["GDP_rank"] / rank_sum
    AQ_weight = user_profile["AQ_rank"] / rank_sum
    
    # Create a new column in the dataframe that combines the weights with the corresponding variables
    df.loc[:, 'weighted_sum'] = (LGBTQ_weight * df['LGBTQ_norm']) + \
                                (WPS_weight * df['WPS_norm']) + \
                                (freedom_weight * df['Freedom_norm']) + \
                                (GDP_weight * df['GDP_norm']) + \
                                (AQ_weight) * df['AQ_norm']
    
    # Find the top n rows with the highest weighted sums
    sorted_df = df.sort_values(by='weighted_sum', ascending=False).reset_index(drop=True)
    n_best = sorted_df[['city', 'country']].head(n).to_records(index=False)  # Use .loc to slice and get a view of the original data
    
    # Return a list of the 'City' values of the top n rows
    return n_best


## Additional Test Cases

In [16]:
user1 = {
    'sector' : "Service",
    'climate' : "Cold",
    'city_size' : "Big City",
    'LGBTQ_rank' : 1,
    'WPSI_rank' : 2,
    'freedom_rank' : 3,
    'GDP_rank' : 4,
    'AQ_rank' : 5
    }
user2 = {
    'sector' : "Industrial",
    'climate' : "Cold",
    'city_size' : "Big City",
    'LGBTQ_rank' : 4 ,
    'WPSI_rank' : 3,
    'freedom_rank' : 2,
    'GDP_rank' : 1,
    'AQ_rank' : 5
    }

user3 = {
    'sector' : "Service",
    'climate' : "Temperate",
    'city_size' : "City",
    'LGBTQ_rank' : 5,
    'WPSI_rank' : 2,
    'freedom_rank' : 3,
    'GDP_rank' : 4,
    'AQ_rank' : 1
    }

user4 = {
    'sector' : "Agricultural",
    'climate' : "Tropical",
    'city_size' : "Rural",
    'LGBTQ_rank' : 5,
    'WPSI_rank' : 1,
    'freedom_rank' : 2,
    'GDP_rank' : 4,
    'AQ_rank' : 1
    }

In [17]:
user1_best = optimize(df, user1, n=5)
print("User1 5 Best places: ", user1_best)

user2_best = optimize(df, user2, n=5)
print("User2 5 Best places: ", user2_best)

user3_best = optimize(df, user3, n=5)
print("User3 5 Best places: ", user3_best)

user4_best = optimize(df, user4, n=5)
print("User1 5 Best places: ", user4_best)

User1 5 Best places:  [('Turku', 'Finland') ('Kuopio', 'Finland') ('Naha', 'Japan')
 ('Helsinki', 'Finland') ('Tampere', 'Finland')]
User2 5 Best places:  [('Seoul', 'South Korea') ('Busan', 'South Korea')
 ('Incheon', 'South Korea') ('Daegu', 'South Korea')
 ('Gwangju', 'South Korea')]
User3 5 Best places:  [('Elbasan', 'Albania') ('Ragusa', 'Italy') ('Agrigento', 'Italy')
 ('Caltanissetta', 'Italy') ('Gela', 'Italy')]
User1 5 Best places:  [('Bolama', 'Guinea-Bissau') ('Fulacunda', 'Guinea-Bissau')
 ('Yakossi', 'Central African Republic')]
